In [1]:
# ! pip install pandas
import pandas as pd

In [81]:
df1 = pd.read_csv('comments_data/comments/df_1.csv', encoding='utf-8')
df2 = pd.read_csv('comments_data/comments/df_2.csv', encoding='utf-8')
df3 = pd.read_csv('comments_data/comments/df_3.csv', encoding='utf-8')
dfw = pd.read_csv('comments_data/comments/df_w.csv', encoding='utf-8')

df = pd.concat([df1, df2, df3, dfw], ignore_index=True)
print(len(df))

160307


### Clean data

In [84]:
def process_merged_comments(df):
    cleaned_rows = []
    i = 0
    total_rows = len(df)

    while i < total_rows:
        row = df.iloc[i]
        title = str(row['video_title']).strip()
        comment = str(row['comment_text']).strip() if pd.notna(row['comment_text']) else ""

        is_main = title.startswith("【#賀瓏夜夜秀】") and comment != ""

        # 如果是主留言且下一行也是主留言 → 無需合併
        if is_main and i + 1 < total_rows:
            next_row = df.iloc[i + 1]
            next_title = str(next_row['video_title']).strip()
            next_comment = str(next_row['comment_text']).strip() if pd.notna(next_row['comment_text']) else ""
            if next_title.startswith("【#賀瓏夜夜秀】") and next_comment != "":
                cleaned_rows.append(row)
                i += 1
                continue

        # 如果不是主留言，跳過
        if not is_main:
            i += 1
            continue

        # 開始合併
        combined_comment = comment
        print(f"==> 第{i}行需要合併")
        meta = {
            'published_at': row['published_at'],
            'author_name': row['author_name'],
            'like_count': row['like_count'],
            'comment_type': row['comment_type']
        }

        j = i + 1
        while j < total_rows:
            continuation_row = df.iloc[j]
            cont_title = str(continuation_row['video_title']).strip()

            # 若遇到下一個主留言就停止
            if cont_title.startswith("【#賀瓏夜夜秀】"):
                break

            print(f"合併第{i}行第{j}行")

            # 如果 continuation 中出現 published_at → 判定為 metadata 錯位行，進行欄位還原，並結束合併
            if pd.notna(continuation_row['published_at']):
                corrected_meta = {
                    'published_at': continuation_row['comment_text'],
                    'author_name': continuation_row['published_at'],
                    'like_count': continuation_row['author_name'],
                    'comment_type': continuation_row['like_count']
                }
                for col, val in corrected_meta.items():
                    if pd.notna(val):
                        meta[col] = val

                if cont_title:
                    combined_comment += cont_title
                j += 1
                break

            # 正常 continuation：只合併 video_title
            if cont_title:
                combined_comment += cont_title
            j += 1

        # 組成新 row，並寫入合併後資料與修正後 metadata
        new_row = row.copy()
        new_row['comment_text'] = combined_comment
        for col in meta:
            new_row[col] = meta[col]

        cleaned_rows.append(new_row)
        i = j  # 跳過合併過的行
    
    print("=== 完成合併 ===")

    return pd.DataFrame(cleaned_rows)


In [85]:
df_cleaned = process_merged_comments(df)

==> 第23行需要合併
合併第23行第24行
合併第23行第25行
合併第23行第26行
==> 第180行需要合併
合併第180行第181行
==> 第300行需要合併
合併第300行第301行
合併第300行第302行
合併第300行第303行
合併第300行第304行
合併第300行第305行
合併第300行第306行
合併第300行第307行
合併第300行第308行
合併第300行第309行
合併第300行第310行
合併第300行第311行
合併第300行第312行
合併第300行第313行
合併第300行第314行
合併第300行第315行
合併第300行第316行
合併第300行第317行
合併第300行第318行
合併第300行第319行
合併第300行第320行
合併第300行第321行
合併第300行第322行
合併第300行第323行
合併第300行第324行
合併第300行第325行
合併第300行第326行
合併第300行第327行
合併第300行第328行
==> 第330行需要合併
合併第330行第331行
合併第330行第332行
合併第330行第333行
合併第330行第334行
==> 第1352行需要合併
合併第1352行第1353行
==> 第1354行需要合併
合併第1354行第1355行
==> 第1356行需要合併
合併第1356行第1357行
合併第1356行第1358行
==> 第1604行需要合併
合併第1604行第1605行
合併第1604行第1606行
==> 第1620行需要合併
合併第1620行第1621行
合併第1620行第1622行
合併第1620行第1623行
合併第1620行第1624行
合併第1620行第1625行
合併第1620行第1626行
合併第1620行第1627行
==> 第1631行需要合併
合併第1631行第1632行
合併第1631行第1633行
合併第1631行第1634行
合併第1631行第1635行
合併第1631行第1636行
==> 第1647行需要合併
合併第1647行第1648行
合併第1647行第1649行
==> 第1652行需要合併
合併第1652行第1653行
合併第1652行第1654行
合併第1652行第1655行
合併第1652行第1656行
==>

In [86]:
len(df_cleaned)

153460

In [87]:
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.to_csv('comments_data/comments/all_comments_cleaned.csv', index=False, encoding='utf-8')

# Count comments

In [89]:
df_group = df_cleaned.groupby(['video_title']).size().reset_index(name='comment_count')
df_group

,video_title,comment_count
0,【#賀瓏夜夜秀】1/20 新聞亂報 EP9｜落選夜夜秀謝票大會,1989
1,【#賀瓏夜夜秀】1/27 新聞亂報 EP10｜新聞自報,13393
2,【#賀瓏夜夜秀】1/6 新聞亂報 EP8｜大選前不演了,1533
3,【#賀瓏夜夜秀】10/28 新聞亂報 EP1｜藍白兩情相悅,1117
4,【#賀瓏夜夜秀】11/18 新聞亂報 EP3｜沒想到有今天吧,1001
5,【#賀瓏夜夜秀】11/25 新聞亂報 EP4｜你也可以不同意啊,1420
6,【#賀瓏夜夜秀】11/4 新聞亂報 EP2｜藍白誰攻誰受？,1172
7,【#賀瓏夜夜秀】12/16 新聞亂報 EP6｜Blue不錄了,2160
8,【#賀瓏夜夜秀】12/30 新聞亂報 EP7｜新北割侯戰,1963
9,【#賀瓏夜夜秀】12/9 新聞亂報 EP5｜財產亂報,1256
